# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
weather_df = pd.read_csv('../WeatherPy/WeatherAPI.csv')
weather_df.head()
weather_df= weather_df.rename(columns={'Temperature':'Temp','Wind MPH':'WindSpeed'})
weather_df.head()

,City Name,Country,Latitude,Longitude,Temp,Humidity,Cloudiness%,WindSpeed
0,Saint George,BM,32.38,-64.68,83.14,70,40,9.17
1,Ushuaia,AR,-54.80,-68.30,33.80,80,40,20.80
2,Quatre Cocos,MU,-20.21,57.76,66.09,88,40,2.24
3,Rikitea,PF,-23.12,-134.97,72.25,75,78,11.12
4,Brae,GB,60.40,-1.35,55.40,93,100,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#pull Lat/Lng as your marks on the gmap
mark_locations = weather_df[['Latitude','Longitude']]
#grab humidity to use as your weight
humid= weather_df['Humidity']

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout,zoom_level=1.5,center=(15,25))

heat_layer = gmaps.heatmap_layer(mark_locations,humid,dissipating=False,max_intensity=100,point_radius=1.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Develop a criteria that you want to narrow down the weather for your vacation
# temperature > 65 & < 80, humidity < 70%, and cloudiness < 50%
crit_temp = (weather_df.Temp > 75) & (weather_df.Temp < 90)
crit_humid= (weather_df.Humidity < 65)
crit_wind= (weather_df.WindSpeed < 10)
weather_criteria = crit_temp & crit_humid & crit_wind

ideal_weather_df = weather_df[weather_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head()

,index,City Name,Country,Latitude,Longitude,Temp,Humidity,Cloudiness%,WindSpeed
0,29,Kaka,TM,37.35,59.61,80.51,21,0,2.30
1,34,Teonthar,IN,24.98,81.65,89.92,56,100,3.98
2,43,Abnūb,EG,27.27,31.15,78.80,39,0,9.17
3,59,Havre,US,48.55,-109.68,89.60,19,1,8.05
4,69,Beaverlodge,CA,55.22,-119.44,86.00,33,20,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"]=""
hotel_df.head()

,index,City Name,Country,Latitude,Longitude,Temp,Humidity,Cloudiness%,WindSpeed,Hotel Name
0,29,Kaka,TM,37.35,59.61,80.51,21,0,2.30,
1,34,Teonthar,IN,24.98,81.65,89.92,56,100,3.98,
2,43,Abnūb,EG,27.27,31.15,78.80,39,0,9.17,
3,59,Havre,US,48.55,-109.68,89.60,19,1,8.05,
4,69,Beaverlodge,CA,55.22,-119.44,86.00,33,20,5.82,


In [41]:
params = {
    'radius': 5000,
    'types':'lodging',
    'keyword': "Hotel",
    'key': gkey
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_address = requests.get(base_url, params=params)
    
    hotel_address = hotel_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing Hotel information...skip.")
        
hotel_df.head()        

Missing Hotel information...skip.
Missing Hotel information...skip.
Missing Hotel information...skip.
Missing Hotel information...skip.
Missing Hotel information...skip.
Missing Hotel information...skip.
Missing Hotel information...skip.


,index,City Name,Country,Latitude,Longitude,Temp,Humidity,Cloudiness%,WindSpeed,Hotel Name
0,29,Kaka,TM,37.35,59.61,80.51,21,0,2.30,
1,34,Teonthar,IN,24.98,81.65,89.92,56,100,3.98,ललई होटल चिल्ला
2,43,Abnūb,EG,27.27,31.15,78.80,39,0,9.17,
3,59,Havre,US,48.55,-109.68,89.60,19,1,8.05,Best Western Plus Havre Inn & Suites
4,69,Beaverlodge,CA,55.22,-119.44,86.00,33,20,5.82,Side Trax Inn


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [45]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…